# 딥 러닝을 이용한 자연어 처리 입문

[구글 코랩에서 실행하기](https://colab.research.google.com/drive/1SUvBCSKYEQri5uaPkqNrMD4oKoYulP2j?usp=sharing)

https://wikidocs.net/33274

02-09 데이터의 분리(Splitting Data)

In [1]:
import tensorflow as tf
import keras
import gensim
import os
import nltk
import sklearn


print(tf.__version__)
print(keras.__version__)
print(gensim.__version__)
print(sklearn.__version__)
print(nltk.__version__)

2.12.0
2.12.0
4.3.1
1.2.2
3.8.1


In [2]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 21.4 MB/s eta 0:00:00


1. 지도 학습(Supervised Learning)

지도 학습의 훈련 데이터는 정답이 무엇인지 맞춰 하는 '문제'에 해당되는 데이터와 레이블이라고 부르는 '정답'이 적혀있는 데이터로 구성\
쉽게 비유하면, 기계는 정답이 적혀져 있는 문제지를 문제와 정답을 함께 보면서 열심히 공부하고, 향후에 정답이 없는 문제에 대해서도 정답을 잘 예측

<훈련 데이터>\
X_train : 문제지 데이터\
y_train : 문제지에 대한 정답 데이터.

<테스트 데이터>\
X_test : 시험지 데이터.\
y_test : 시험지에 대한 정답 데이터.

X_train과 y_train에 대해서 학습을 합니다. \
기계는 학습 상태에서는 정답지인 y_train을 볼 수 있기 때문에 18,000개의 문제지 X_train과 y_train을 함께 보면서 어떤 메일 내용일 때 정상 메일인지 스팸 메일인지를 열심히 규칙을 도출해나가면서 정리해나갑니다. \
그리고 학습을 다 한 기계에게 y_test는 보여주지 않고, X_test에 대해서 정답을 예측하게 합니다. 그리고 기계가 예측한 답과 실제 정답인 y_test를 비교하면서 기계가 정답을 얼마나 맞췄는지를 평가합니다. 이 수치가 기계의 정확도(Accuracy)가 됩니다.

2. X와 y분리하기

1) zip 함수를 이용하여 분리하기

In [3]:
X, y = zip(['a', 1], ['b', 2], ['c', 3])
print('X 데이터 :', X)
print('y 데이터 :', y)

X 데이터 : ('a', 'b', 'c')
y 데이터 : (1, 2, 3)


각 데이터에서 첫번째로 등장한 원소들끼리 묶이고, 두번째로 등장한 원소들끼리 묶인 것을 볼 수 있습니다.

In [5]:
# 2D 텐서.
sequences = [['a', 1], ['b', 2], ['c', 3]]
X, y = zip(*sequences)
print('X 데이터 :', X)
print('y 데이터 :', y)

X 데이터 : ('a', 'b', 'c')
y 데이터 : (1, 2, 3)


2) 데이터프레임을 이용하여 분리하기

In [7]:
import pandas as pd

In [8]:
values = [['당신에게 드리는 마지막 혜택!', 1],
          ['내일 뵐 수 있을지 확인 부탁드...', 0],
          ['도연씨. 잘 지내시죠? 오랜만입...', 0],
          ['(광고) AI로 주가를 예측할 수 있다!' ,1]]
columns = ['메일 본문', '스팸 메일 유무']

df = pd.DataFrame(values, columns=columns)
df

,메일 본문,스팸 메일 유무
0,당신에게 드리는 마지막 혜택!,1
1,내일 뵐 수 있을지 확인 부탁드...,0
2,도연씨. 잘 지내시죠? 오랜만입...,0
3,(광고) AI로 주가를 예측할 수 있다!,1


데이터프레임은 열의 이름으로 각 열에 접근이 가능하므로, 이를 이용하면 손쉽게 X 데이터와 y 데이터를 분리

In [9]:
X = df['메일 본문']
y = df['스팸 메일 유무']

In [10]:
print('X 데이터 :', X.to_list())
print('y 데이터 :', y.to_list())

X 데이터 : ['당신에게 드리는 마지막 혜택!', '내일 뵐 수 있을지 확인 부탁드...', '도연씨. 잘 지내시죠? 오랜만입...', '(광고) AI로 주가를 예측할 수 있다!']
y 데이터 : [1, 0, 0, 1]


3) Numpy를 이용하여 분리하기

임의의 데이터를 만들어서 Numpy의 슬라이싱(slicing)을 사용하여 데이터를 분리

In [12]:
import numpy as np

In [13]:
np_array = np.arange(0, 16).reshape((4, 4))
print('전체 데이터 :')
print(np_array)

전체 데이터 :
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]


마지막 열을 제외하고 X데이터에 저장, 마지막 열만을 y데이터에 저장

In [14]:
X = np_array[:, :3]
y = np_array[:, 3]

print('X 데이터 :')
print(X)
print('y 데이터 :', y)

X 데이터 :
[[ 0  1  2]
 [ 4  5  6]
 [ 8  9 10]
 [12 13 14]]
y 데이터 : [ 3  7 11 15]


3. 테스트 데이터 분리하기

 X와 y가 분리된 데이터에 대해서 테스트 데이터를 분리하는 과정

1) 사이킷 런을 이용하여 분리

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

X : 독립 변수 데이터. (배열이나 데이터프레임)\
y : 종속 변수 데이터. 레이블 데이터.\
test_size : 테스트용 데이터 개수를 지정한다. \
1보다 작은 실수를 기재할 경우, 비율을 나타낸다.\
train_size : 학습용 데이터의 개수를 지정한다. \
1보다 작은 실수를 기재할 경우, 비율을 나타낸다.\
random_state : 난수 시드

임의로 X 데이터와 y 데이터를 생성

In [17]:
X, y = np.arange(10).reshape((5, 2)), range(5)

print('X 전체 데이터 :')
print(X)
print('y 전체 데이터 :')
print(list(y))

X 전체 데이터 :
[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
y 전체 데이터 :
[0, 1, 2, 3, 4]


 7:3의 비율로 데이터를 분리\
  train_test_split()은 기본적으로 데이터의 순서를 섞고나서 훈련 데이터와 테스트 데이터를 분리\
  random_state의 값을 특정 숫자로 기재해준 뒤에 다음에도 동일한 숫자로 기재해주면 항상 동일한 훈련 데이터와 테스트 데이터를 얻을 수 있습니다.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1234)

70%의 비율로 분리된 X의 훈련 데이터와 30%의 비율로 분리된 X의 테스트 데이터

In [19]:
print('X 훈련 데이터 :')
print(X_train)
print('X 테스트 데이터 :')
print(X_test)

X 훈련 데이터 :
[[2 3]
 [4 5]
 [6 7]]
X 테스트 데이터 :
[[8 9]
 [0 1]]


In [20]:
print('y 훈련 데이터 :')
print(y_train)
print('y 테스트 데이터 :')
print(y_test)

y 훈련 데이터 :
[1, 2, 3]
y 테스트 데이터 :
[4, 0]


In [22]:
# random_state의 값을 변경
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

print('y 훈련 데이터 :')
print(y_train)
print('y 테스트 데이터 :')
print(y_test)

y 훈련 데이터 :
[4, 0, 3]
y 테스트 데이터 :
[2, 1]


random_state 값이 1234일 때와 전혀 다른 y데이터가 출력된다.\
데이터가 다른 순서로 섞였다는 의미.

In [24]:
# random_state의 값을 변경
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1234)

print('y 훈련 데이터 :')
print(y_train)
print('y 테스트 데이터 :')
print(y_test)

y 훈련 데이터 :
[1, 2, 3]
y 테스트 데이터 :
[4, 0]


이전과 동일한 y데이터가 출력, \
random_state의 값을 고정해두면 실행할 때마다 항상 동일한 순서로 데이터를 섞으므로, 동일한 코드를 다음에 재현하고자 할 때 사용

2) 수동으로 분리하기

데이터를 분리하는 방법 중 하나는 수동으로 분리하는 것

In [25]:
X, y = np.arange(0 ,24).reshape((12, 2)), range(12)

print('X 전체 데이터 :')
print(X)
print('y 전체 데이터 :')
print(list(y))

X 전체 데이터 :
[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]
 [14 15]
 [16 17]
 [18 19]
 [20 21]
 [22 23]]
y 전체 데이터 :
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [26]:
num_of_train = int(len(X) * 0.8) # 데이터 전체 길이의 80%에 해당하는 길이값을 구한다.
num_of_test = int(len(X) - num_of_train) # 전체 길이에서 80%에 해당하는 길이를 뺀다.
print('훈련 데이터의 크기 :', num_of_train)
print('테스트 데이터의 크기 :', num_of_test)

훈련 데이터의 크기 : 9
테스트 데이터의 크기 : 3


아직 훈련 데이터와 테스트 데이터를 나눈 것이 아니라 이 두 개의 개수를 몇 개로 할지 정하기만 한 상태입니다. 여기서 num_of_test를 len(X) * 0.2로 계산해서는 안 됩니다. 데이터에 누락이 발생할 수 있습니다. 예를 들어서 전체 데이터의 개수가 4,518이라고 가정했을 때 4,518의 80%의 값은 3,614.4로 소수점을 내리면 3,614가 됩니다. 또한 4,518의 20%의 값은 903.6으로 소수점을 내리면 903이 됩니다. 그리고 3,614 + 903 = 4517이므로 데이터 1개가 누락이 됩니다. 그러므로 어느 한 쪽을 먼저 계산하고 그 값만큼 제외하는 방식으로 계산해야 합니다.

In [27]:
X_test = X[num_of_train:] # 전체 데이터 중에서 20%만큼 뒤의 데이터 저장
y_test = y[num_of_train:] # 전체 데이터 중에서 20%만큼 뒤의 데이터 저장
X_train = X[:num_of_train] # 전체 데이터 중에서 80%만큼 앞의 데이터 저장
y_train = y[:num_of_train] # 전체 데이터 중에서 80%만큼 앞의 데이터 저장

데이터를 나눌 때는 num_of_train와 같이 하나의 변수만 사용하면 데이터의 누락을 방지

In [28]:
print('X 테스트 데이터 :')
print(X_test)
print('y 테스트 데이터 :')
print(list(y_test))

X 테스트 데이터 :
[[18 19]
 [20 21]
 [22 23]]
y 테스트 데이터 :
[9, 10, 11]


train_test_split()과 다른 점은 데이터가 섞이지 않은 채 어느 지점에서 데이터를 앞과 뒤로 분리했다는 점\
만약, 수동으로 분리하게 된다면 데이터를 분리하기 전에 수동으로 데이터를 섞는 과정이 필요할 수 있습니다.